# Adventure of code 2020

Adventure of code [2020](https://adventofcode.com/2020) in a monolitic notebook.

In [1]:
from __future__ import annotations
from math import prod
from collections import namedtuple, Counter, deque, defaultdict
from itertools import combinations, product
from string import ascii_lowercase, digits
from dataclasses import dataclass, field
from typing import Tuple, Optional, NamedTuple

In [2]:
def read_input(path, parse_int=False):
    with open(path) as f:
        for _, line in enumerate(f):

            line = line.strip()
            if parse_int:
                line = int(line)

            yield line

## Day 1

Find the sum equal to the target value.

In [3]:
data = list(read_input("inputs/input_01.txt", parse_int=True))
target = 2020

for title, count in zip(["a", "b"], [2, 3]):
    result = [
        prod(items) for items in combinations(data, count) if sum(items) == target
    ][0]
    print(f"{title} = {result}")

a = 1019904
b = 176647680


## Day 2

Check if a policy (an str) is valid for a given condition.

In [4]:
Policy = namedtuple("policy", ["number_0", "number_1", "letter", "policy"])


def parse_policy(raw_data):
    for raw_item in raw_data:
        rest, policy_str = raw_item.split(":")
        rest, letter = rest.split(" ")
        number_0, number_1 = tuple(map(int, rest.split("-")))
        policy_str = policy_str.strip()

        yield Policy(number_0, number_1, letter, policy_str)


def check_policy_1(policy: Policy) -> bool:
    """Check if the policy count for the 'letter' is inside de numbers range."""
    count = Counter(policy.policy)
    count_letter = count.get(policy.letter, 0)

    if policy.number_0 <= count_letter <= policy.number_1:
        return True
    else:
        return False


def check_policy_2(policy: Policy) -> bool:
    """Check if there is the correct 'letter' in the numbers positions.
    Positions start counting from 1, so we sustract 1."""
    return (
        sum(
            policy.policy[i - 1] == policy.letter
            for i in [policy.number_0, policy.number_1]
        )
        == 1
    )


raw_data = read_input("inputs/input_02.txt")
result = sum(check_policy_1(policy) for policy in parse_policy(raw_data))
print(f"a = {result}")

raw_data = read_input("inputs/input_02.txt")
result = sum(check_policy_2(policy) for policy in parse_policy(raw_data))
print(f"b = {result}")

a = 506
b = 443


## Day 3

## Day 4
Check if a passport (a dict) is valid

In [5]:
valid_keys = ["byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid", "cid"]


def parse_passports(raw_data):
    passport = {}
    for line in raw_data:
        if not line:
            yield passport
            passport = {}

        else:
            for raw_field in line.split():
                k, v = raw_field.split(":")
                passport[k] = v
    # return the last one
    yield passport


def check_passport_1(passport: dict) -> bool:
    """Check if the passport have all the keys (the last one is optional)"""
    return all(k in passport for k in valid_keys[:-1])


def check_generic_field(raw_field: str, low_value: int, upper_value: int) -> bool:
    try:
        field = int(raw_field)
    except ValueError:
        return False

    if len(raw_field) != 4:
        return False

    if not low_value <= field <= upper_value:
        return False

    return True


def check_hgt_field(raw_field: str) -> bool:
    try:
        field = int(raw_field[:-2])
    except ValueError:
        return False

    field_type = raw_field[-2:]

    if field_type not in ["cm", "in"]:
        return False
    elif field_type == "cm":
        low_value = 150
        upper_value = 193
    elif field_type == "in":
        low_value = 59
        upper_value = 76

    if not low_value <= field <= upper_value:
        return False

    return True


def check_hcl_field(raw_field: str) -> bool:
    if raw_field[0] != "#":
        return False

    for char in raw_field[1:]:
        if char not in digits + ascii_lowercase[:6]:
            return False
    return True


def check_ecl_field(raw_field: str) -> bool:
    return raw_field in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]


def check_pid_field(raw_field: str) -> bool:
    if len(raw_field) != 9:
        return False

    return raw_field.isnumeric()


def check_passport_2(passport: dict) -> bool:
    """Check that each passport field is valid."""
    if not all(k in passport for k in valid_keys[:-1]):
        return False

    conditions = [
        check_generic_field(passport["byr"], 1920, 2002),
        check_generic_field(passport["iyr"], 2010, 2020),
        check_generic_field(passport["eyr"], 2020, 2030),
        check_hgt_field(passport["hgt"]),
        check_hcl_field(passport["hcl"]),
        check_ecl_field(passport["ecl"]),
        check_pid_field(passport["pid"]),
    ]

    return all(conditions)


raw_data = read_input("inputs/input_04.txt")
result = sum(map(check_passport_1, parse_passports(raw_data)))
print(f"a = {result}")

raw_data = read_input("inputs/input_04.txt")
result = sum(map(check_passport_2, parse_passports(raw_data)))
print(f"b = {result}")

a = 219
b = 127


## Day 5

In [6]:
def parse_seat(seat: str) -> Tuple[int, int]:
    row = int(seat[:-3].replace("F", "0").replace("B", "1"), 2)
    column = int(seat[-3:].replace("L", "0").replace("R", "1"), 2)
    return row, column


def compute_seat_id(row: int, column: int) -> int:
    return row * 8 + column


def find_missing_seat(all_seats) -> int:
    seats_sorted = sorted(all_seats)
    current_seat = seats_sorted[0]
    for seat in seats_sorted[1:]:
        if seat - current_seat != 1:
            return current_seat + 1  # or "seat - 1"

        current_seat = seat


raw_data = read_input("inputs/input_05.txt")
result = max(
    compute_seat_id(row, column) for row, column in (map(parse_seat, raw_data))
)
print(f"a = {result}")

raw_data = read_input("inputs/input_05.txt")
result = find_missing_seat(
    compute_seat_id(row, column) for row, column in (map(parse_seat, raw_data))
)
print(f"b = {result}")

a = 959
b = 527


## Day 6

## Day 7

## Day 8

In [7]:
Instruction = namedtuple("Instruction", ["operation", "value"])


def parse_instruction(raw_instruction: str) -> Instruction:
    operation, value = raw_instruction.split()
    return Instruction(operation, int(value))


def compute_instruction(instruction, acc, pos):
    if instruction.operation == "acc":
        acc += instruction.value
        pos += 1
    elif instruction.operation == "jmp":
        pos += instruction.value
    elif instruction.operation == "nop":
        pos += 1

    return acc, pos


def run_program(all_instructions):
    """Compute the instructions.
    Return:
        acc: the final acc value
        end_type: False if is an early stop (hit an know instruction), else True
    """
    acc = 0
    pos = 0

    know_pos = set()

    while pos not in know_pos:
        know_pos.add(pos)
        instruction = all_instructions[pos]
        acc, pos = compute_instruction(instruction, acc, pos)
        if pos == len(all_instructions):
            return acc, True

    return acc, False


def find_invalid_instruction(all_instructions):
    """Change one instruction at time and check if program finish with no problems."""
    for i, instruction in enumerate(all_instructions):
        if instruction.operation == "acc":
            continue
        elif instruction.operation == "jmp":
            new_operator = "nop"
        elif instruction.operation == "nop":
            new_operator = "jmp"

        all_instructions_copy = all_instructions.copy()
        all_instructions_copy[i] = Instruction(new_operator, instruction.value)

        acc, end_ok = run_program(all_instructions_copy)
        if end_ok:
            return acc


raw_data = read_input("inputs/input_08.txt")
all_instructions = list(map(parse_instruction, raw_data))
result = run_program(all_instructions)[0]
print(f"a = {result}")

result = find_invalid_instruction(all_instructions)
print(f"b = {result}")

a = 1200
b = 1023


## Day 9

In [8]:
def find_invalid_number(numbers):
    # "-25 because we are not getting the last number"
    for i in range(len(numbers) - 25):
        target = numbers[i + 25]
        possibles_sums = map(sum, combinations(numbers[i : i + 25], 2))
        if numbers[i + 25] not in possibles_sums:
            return target


def find_encryption_weakness(numbers, target):
    n = 2
    while True:
        for i in range(len(numbers) - n - 1):
            candidates = numbers[i : i + n]
            if sum(candidates) == target:
                return max(candidates) + min(candidates)
        n += 1


raw_data = read_input("inputs/input_09.txt", parse_int=True)
numbers = list(raw_data)
result = find_invalid_number(numbers)
print(f"a = {result}")

result = find_encryption_weakness(numbers, result)
print(f"b = {result}")

a = 1930745883
b = 268878261


## Day 10

In [9]:
def preprocess_adapters(raw_data):
    """Sort and add the first and last element to data."""
    # add the first adapter, always zero
    all_adapter = [0] + sorted(raw_data)
    # add the last one, three more that the last one
    all_adapter += [all_adapter[-1] + 3]
    return all_adapter


def diff_counter_elem(elements):
    """Return the counter of the difference of adjacent elements."""
    return Counter(elements[i + 1] - elements[i] for i in range(len(elements) - 1))


def multiply_count_adapter_jumps(raw_data):
    count_jumps = diff_counter_elem(all_adapter)
    return count_jumps[1] * count_jumps[3]


def find_chunks(all_adapter):
    chunks = []
    chunk = []
    for i in range(len(all_adapter) - 1):
        chunk.append(all_adapter[i])

        if all_adapter[i + 1] - all_adapter[i] == 3:
            chunks.append(chunk)
            chunk = []

    if chunk:
        chunks.append(chunk)

    return chunks


def count_cases_in_chunk(chunk):
    """Look how many combinations are valid for a chunk."""
    valid_cases = 1
    for cases_len in range(2, len(chunk)):
        for case in combinations(chunk, cases_len):
            # the fist and last element must be in the case
            if chunk[0] not in case or chunk[-1] not in case:
                continue

            count_jumps = diff_counter_elem(case)
            # only can jump 
            if any(i > 3 for i in count_jumps.keys()):
                continue

            valid_cases += 1
    return valid_cases


def count_valid_cases(all_adapter):
    chunks = find_chunks(all_adapter)
    #
    chunks = [chunk for chunk in chunks if len(chunk) > 2]
    return prod(map(count_cases_in_chunk, chunks))


all_adapter = preprocess_adapters(read_input("inputs/input_10.txt", parse_int=True))
result = multiply_count_adapter_jumps(all_adapter)
print(f"a = {result}")

result = count_valid_cases(all_adapter)
print(f"b = {result}")

a = 2368
b = 1727094849536


## Day 11

## Day 12

In [10]:
cardinal_letters = "ENWS"

@dataclass
class Point:
    x: int
    y: int

    def add(self, other_point: Point):
        self.x += other_point.x
        self.y += other_point.y
        
    def scale(self, n: int) -> Point:
        return Point(self.x*n, self.y*n)
    
@dataclass
class Ferry:
    pos: Point
    direction: deque
    
    def move(self, move_direction: Point, number: int):
        self.pos.add(move_direction.scale(number))
    
    def rotate(self, letter: str, number: int):
        n = number // 90
        if letter == "L":
            n *= -1

        self.direction.rotate(n)    
            
    
def parse_instruction_part_1(instruction, ferry):
    letter, number = instruction[0], int(instruction[1:])
    if letter in cardinal_letters:
        move_direction = direction_to_pos[letter]
        ferry.move(move_direction, number)
            
    elif letter in "LR":
        ferry.rotate(letter, number)
    elif letter == "F":
        move_direction = direction_to_pos[ferry.direction[0]]
        ferry.move(move_direction, number)

        
@dataclass
class Waypoint:
    pos: Point
    ship: Point
    
    def move(self, move_direction: Point, number: int):
        self.pos.add(move_direction.scale(number))
    
    def rotate(self, letter: str, number: int):
        n = number // 90
        for _ in range(n):
            if letter == "R":
                self.pos = Point(self.pos.y, -self.pos.x)
            elif letter == "L":
                self.pos = Point(-self.pos.y, self.pos.x)

    def move_ship(self, n: int):
        for _ in range(n):
            self.ship.add(self.pos)
            
def parse_instruction_part_2(instruction, waypoint):
    letter, number = instruction[0], int(instruction[1:])
    if letter in cardinal_letters:
        move_direction = direction_to_pos[letter]
        waypoint.move(move_direction, number)
            
    elif letter in "LR":
        waypoint.rotate(letter, number)
        
    elif letter == "F":
        waypoint.move_ship(number)
        
direction_to_pos = {"E": [1, 0], "N":[0, 1], "W":[-1, 0], "S":[0,-1]}
direction_to_pos = {k:Point(*v) for k, v in direction_to_pos.items()}

instructions = list(read_input("inputs/input_12.txt"))
ferry = Ferry(Point(0,0), deque(list("ENWS"))) 

for instruction in instructions:
    parse_instruction_part_1(instruction, ferry)
result = sum(abs(i) for i in [ferry.pos.x, ferry.pos.y])
print(f"a = {result}")

waypoint = Waypoint(Point(10, 1), Point(0, 0))
for instruction in instructions:
    parse_instruction_part_2(instruction, waypoint)
    
result = sum(abs(i) for i in [waypoint.ship.x, waypoint.ship.y])
print(f"b = {result}")

a = 759
b = 45763


## Day 13

In [11]:
raw_data = list(read_input("inputs/input_13.txt"))
early_timestamp = int(raw_data[0])
raw_buses = raw_data[1].split(",")
buses = [int(i) for i in raw_buses if i != "x"]

def find_min_multiple(n: int, threshold: int) -> int:
    value = n
    while value <= threshold:
        value += n
        
    return value

stop_buses_min = {i:find_min_multiple(i, early_timestamp) for i in buses}
early_bus = min(stop_buses_min, key=stop_buses_min.get)

result = early_bus*(stop_buses_min[early_bus] - early_timestamp)
print(f"a = {result}")

# TODO Part b

a = 3215


## Day 14

In [12]:
from itertools import zip_longest

class Mask(NamedTuple):
    mask: str
    kv_list: List[KV]

class KV(NamedTuple):
    key: int
    value: int
    
def parse_data(raw_data):

    data = []
    mask = ""

    for line in raw_data:
        if line.startswith("mask"):
            if mask:
                data.append(Mask(mask, kv_list))

            mask = line[7:]
            kv_list = []
        else:
            raw_key, raw_value = line.split(" = ")
            key, value = int(raw_key[4:-1]), int(raw_value)
            kv_list.append(KV(key, value))
            
    # add the last one
    data.append(Mask(mask, kv_list))
    return data

def compare_single_value(i_value: Optional[str], i_mask: str) -> str:
    if not i_value and i_mask.isdigit():
        return i_mask
    elif not i_value and i_mask == "X":
        return "0"
    elif i_value.isdigit() and i_mask.isdigit():
        return i_mask
    elif i_value.isdigit() and i_mask == "X":
        return i_value
    else:
        raise ValueError(f"invalid values {i_value=}, {i_mask=}")
    
def apply_mask_to_value(b_value: str, mask: str) -> str:
    result = []
    for i_value, i_mask in zip_longest(reversed(b_value), reversed(mask)):
        result.append(compare_single_value(i_value, i_mask))
    return "".join(result)[::-1]


def sum_part_a(data):
    mem = dict()
    for row in data:
        for kv in row.kv_list:
            b_value = bin(kv.value)[2:]
            mem[kv.key] = apply_mask_to_value(b_value, row.mask)
    return sum(int(i, base=2) for i in mem.values())

def compare_single_key(i_value: Optional[str], i_mask: str) -> str:
    if i_mask in ["X", "1"]:
        return i_mask
    elif i_mask == "0":
        if i_value:
            return i_value
        else:
            return "0"
    else:
        raise ValueError(f"invalid values {i_value=}, {i_mask=}")

def apply_mask_to_key(b_key: str, mask: str) -> List[int]:
    key_with_mask = []
    for i_key, i_mask in zip_longest(reversed(b_key), reversed(mask)):
        key_with_mask.append(compare_single_key(i_key, i_mask))
    key_with_mask = "".join(key_with_mask)[::-1]

    results = []
    for values_to_change in product(["0", "1"], repeat=mask.count("X")):
        result = key_with_mask
        for i in values_to_change:
            result = result.replace("X", i, 1)
        results.append(int(result, base=2))
        
    return results

def sum_part_b(data):
    mem = dict()
    for row in data:
        for kv in row.kv_list:
            b_key = bin(kv.key)[2:]
            for k in apply_mask_to_key(b_key, row.mask):
                mem[k] = kv.value

    return sum(i for i in mem.values())

data = parse_data(read_input("inputs/input_14.txt"))
result = sum_part_a(data)
print(f"a = {result}")
result = sum_part_b(data)
print(f"b = {result}")

a = 15514035145260
b = 3926790061594
